In [2]:
import sys
import spotipy
import spotipy.util as util


In [3]:
username = 'Franzisko'
scope = 'user-library-read playlist-modify-private'

In [33]:
#É necessário se cadastrar no spotify para obter os dados de acesso a API
token = util.prompt_for_user_token(username,
                                   scope,
                                   client_id=' ',
                                   client_secret=' ',
                                   redirect_uri='http://localhost:8080/callback')

In [34]:
offset = 0
songs = []
items = []
ids = []

In [35]:
def getTrackFeatures(ids,artist_uri): # Obtendo as features
  meta = sp.track(ids)
  features = sp.audio_features(ids)
  artist_info = sp.artist(artist_uri)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  duration_ms = meta['duration_ms']
  popularity = meta['popularity']
    
  # Artist 
  artist_pop = artist_info['popularity']
  artist_genres = artist_info['genres']
    
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [ids, name, album, artist, artist_genres, release_date,artist_pop , duration_ms, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

In [36]:
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    while True: #Obtendo as informações de cada música
        content = sp.current_user_saved_tracks(limit=50, offset=offset)
        songs += content['items']
        if content['next'] is not None:
            offset += 100
        else:
            break
            
    for i in songs: # Obtendo os ids da músicas
        ids.append(i['track']['id'])
    
    artist=[]
    for i in songs: # Obtendo os ids das bandas/cantores
        artist.append(i['track']['artists'][0]['uri'])
    import time

    tracks = []
    for i in range(len(ids)):
        #time.sleep(.5)
        track = getTrackFeatures(ids[i],artist[i])# Crindo o dataset com as caracteristicas de cada música
        tracks.append(track)
else:
    print("Can't get token for", username)


In [38]:
# Salvando em arquivo CSV
import pandas as pd
df = pd.DataFrame(tracks, columns = ['Id','Nome', 'album', 'Banda', 'Genero', 'Data_lancamento','Popularidade_Banda', 'Duracao', 'Popularidade', 'acustica', 'dancabilidade', 'energia', 'Instrumentalidade', 'vivacidade', 'Volume', 'speechiness', 'tempo', 'time_signature'])
df.to_csv("spotify.csv", sep = ',')